# Full Fine-Tuning with Unsloth: Qwen2.5-7B on AG News

This notebook demonstrates **Full Fine-Tuning** using **Unsloth's FastLanguageModel** for optimized performance on DGX Spark.

## Overview

| Aspect | Details |
|--------|---------|
| **Model** | unsloth/Qwen2.5-7B-Instruct |
| **Method** | Full Fine-Tuning (100% parameters) |
| **Framework** | Unsloth + TRL |
| **Dataset** | AG News (120K train, 7.6K test) |
| **Task** | 4-class text classification |
| **Expected Time** | ~6-8 hours (DGX Spark optimized) |
| **Memory** | ~60-70 GB |
| **Output** | Full model (~14 GB) |

## Base Model Performance (Target to Beat)

| Metric | Base Model | Target |
|--------|------------|--------|
| **Accuracy** | 78.76% | >88% |
| **F1 (macro)** | 77.97% | >85% |
| **Sci/Tech F1** | 62.06% | >80% |
| **Business Precision** | 63.66% | >80% |

## Full Fine-Tuning vs LoRA

| Aspect | Full Fine-Tuning | LoRA |
|--------|------------------|------|
| Parameters trained | 7.6B (100%) | ~40M (0.5%) |
| Memory usage | ~60-70 GB | ~25 GB |
| Output size | ~14 GB | ~180 MB |
| Quality | Maximum | Slightly lower |

## DGX Spark Optimizations Applied

| Optimization | Setting | Reason |
|--------------|---------|--------|
| `gradient_checkpointing` | False | Faster training (128GB memory available) |
| `BATCH_SIZE` | 8 | Larger batches for better GPU utilization |
| `GRADIENT_ACCUMULATION` | 2 | Keeps effective batch size = 16 |
| `optim` | adamw_8bit | Reduces optimizer memory by 50% |
| `packing` | True | Efficient sequence packing |
| `dataloader_num_workers` | 8 | Faster parallel data loading |

## Prerequisites

```bash
./start_docker.sh start finetune
# Then open http://localhost:8888
```

**Note**: Full fine-tuning requires significant memory. DGX Spark's 128 GB unified memory is sufficient.

## 1. Environment Setup

In [1]:
import torch
import os

print("=" * 60)
print("Environment Verification - Full Fine-Tuning")
print("=" * 60)

print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Compute Capability: {torch.cuda.get_device_capability(0)}")
    try:
        total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"GPU Memory: {total_mem:.1f} GB")
    except:
        print("GPU Memory: Unified memory system (DGX Spark)")
else:
    raise RuntimeError("CUDA not available!")

print(f"\nWorking directory: {os.getcwd()}")
print(f"Dataset available: {os.path.exists('/fine-tuning/datasets/train.jsonl')}")

Environment Verification - Full Fine-Tuning

PyTorch version: 2.10.0a0+b558c986e8.nv25.11
CUDA available: True
CUDA version: 13.0
GPU: NVIDIA GB10
GPU Compute Capability: (12, 1)
GPU Memory: 128.5 GB

Working directory: /fine-tuning
Dataset available: True


## 2. Configuration

In [2]:
# =============================================================================
# Model Configuration
# =============================================================================
MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct"
MAX_SEQ_LENGTH = 512

# =============================================================================
# Training Configuration (Optimized for DGX Spark 128GB)
# =============================================================================
BATCH_SIZE = 8  # Increased from 4 (DGX Spark has 128GB memory)
GRADIENT_ACCUMULATION_STEPS = 2  # Reduced to keep effective batch size = 16
LEARNING_RATE = 2e-5  # Lower than LoRA for full fine-tuning
NUM_EPOCHS = 1
WARMUP_RATIO = 0.03
WEIGHT_DECAY = 0.01

# =============================================================================
# Output Configuration
# =============================================================================
OUTPUT_DIR = "./checkpoints/qwen7b-ag-news-full-unsloth"
LOGGING_STEPS = 50
SAVE_STEPS = 500

TRAIN_DATA_PATH = "/fine-tuning/datasets/train.jsonl"

print("Full Fine-Tuning Configuration loaded!")
print(f"  Model: {MODEL_NAME}")
print(f"  Method: Full Fine-Tuning (100% parameters)")
print(f"  Batch size: {BATCH_SIZE} x {GRADIENT_ACCUMULATION_STEPS} = {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Output: {OUTPUT_DIR}")

Full Fine-Tuning Configuration loaded!
  Model: unsloth/Qwen2.5-7B-Instruct
  Method: Full Fine-Tuning (100% parameters)
  Batch size: 8 x 2 = 16
  Learning rate: 2e-05
  Output: ./checkpoints/qwen7b-ag-news-full-unsloth


## 3. Load Model with Full Fine-Tuning Enabled

Using `full_finetuning=True` to train all parameters.

In [3]:

from unsloth import FastLanguageModel

print("Loading model for Full Fine-Tuning...")
print(f"  Model: {MODEL_NAME}")
print(f"  This will train ALL 7.6B parameters!")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect (will use BF16)
    load_in_4bit=False,  # Full fine-tuning uses full precision
    full_finetuning=True,  # ← KEY: Enable full fine-tuning
    use_exact_model_name=True,  # Use cached model, don't look for alternatives
)

mem_used = torch.cuda.memory_allocated() / 1e9

print(f"\n✓ Model loaded for full fine-tuning!")
print(f"  GPU memory used: {mem_used:.2f} GB")
print(f"  Tokenizer vocab size: {len(tokenizer)}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.nash_md_trainer: Failed to import trl.trainer.nash_md_trainer because of the following error (look up to see its traceback):
cannot import name 'amp' from 'apex' (/usr/local/lib/python3.12/dist-packages/apex/__init__.py)
Unsloth: Could not import trl.trainer.online_dpo_trainer: Failed to import trl.trainer.online_dpo_trainer because of the following error (look up to see its traceback):
cannot import name 'amp' from 'apex' (/usr/local/lib/python3.12/dist-packages/apex/__init__.py)
Unsloth: Could not import trl.trainer.xpo_trainer: Failed to import trl.trainer.xpo_trainer because of the following error (look up to see its traceback):
cannot import name 'amp' from 'apex' (/usr/local/lib/python3.12/dist-packages/apex/__init__.py)
Loading model for Full Fine-Tuning...
  Model: unsloth/Qwen2.5-7B-Instruct
  This will train ALL 7.6B parameters!
==((====))==  Unsloth 2026.1.4: Fast Qwen2 pat

Loading checkpoint shards: 100%|██████████| 4/4 [01:17<00:00, 19.26s/it]



✓ Model loaded for full fine-tuning!
  GPU memory used: 15.23 GB
  Tokenizer vocab size: 151665


## 4. Verify All Parameters Are Trainable

In [4]:
# Count parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print(f"Parameter Count:")
print(f"  Trainable: {trainable:,} ({100*trainable/total:.2f}%)")
print(f"  Total: {total:,}")

if trainable == total:
    print(f"\n✓ All parameters are trainable (Full Fine-Tuning mode)")
else:
    print(f"\n⚠ Warning: Only {100*trainable/total:.2f}% of parameters are trainable")

Parameter Count:
  Trainable: 7,615,616,512 (100.00%)
  Total: 7,615,616,512

✓ All parameters are trainable (Full Fine-Tuning mode)


## 5. Load Training Dataset

In [5]:
from datasets import load_dataset

print(f"Loading dataset from: {TRAIN_DATA_PATH}")

dataset = load_dataset("json", data_files=TRAIN_DATA_PATH, split="train")

print(f"\nDataset loaded:")
print(f"  Total examples: {len(dataset):,}")
print(f"  Columns: {dataset.column_names}")

print(f"\nSample entry:")
sample = dataset[0]
for msg in sample["messages"]:
    role = msg["role"]
    content = msg["content"][:80] + "..." if len(msg["content"]) > 80 else msg["content"]
    print(f"  [{role}]: {content}")

Loading dataset from: /fine-tuning/datasets/train.jsonl

Dataset loaded:
  Total examples: 120,000
  Columns: ['messages']

Sample entry:
  [system]: You are a news article classifier. Your task is to categorize news articles into...
  [user]: Classify the following news article:

Thirst, Fear and Bribes on Desert Escape f...
  [assistant]: {"category":"World"}


## 6. Format Dataset

In [6]:
def formatting_prompts_func(examples):
    """Format examples using the tokenizer's chat template."""
    texts = []
    for messages in examples["messages"]:
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )
        texts.append(text)
    return {"text": texts}

print("Applying chat template to dataset...")
formatted_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    num_proc=4,
    desc="Formatting",
)

print(f"\nFormatted dataset columns: {formatted_dataset.column_names}")
print(f"\nSample (first 400 chars):")
print(formatted_dataset[0]["text"][:400])

Applying chat template to dataset...

Formatted dataset columns: ['messages', 'text']

Sample (first 400 chars):
<|im_start|>system
You are a news article classifier. Your task is to categorize news articles into exactly one of four categories:

- World: News about politics, government, elections, diplomacy, conflicts, and public affairs (domestic or international)
- Sports: News about athletic events, games, players, teams, coaches, tournaments, and championships
- Business: News about companies, markets, f


## 7. Configure Training

In [7]:
from trl import SFTTrainer, SFTConfig

total_steps = (len(formatted_dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)) * NUM_EPOCHS

print(f"Training configuration:")
print(f"  Total examples: {len(formatted_dataset):,}")
print(f"  Batch size: {BATCH_SIZE} x {GRADIENT_ACCUMULATION_STEPS} = {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  Estimated total steps: {total_steps:,}")

sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    optim="adamw_8bit",
    bf16=True,
    fp16=False,
    max_length=MAX_SEQ_LENGTH,
    packing=True,
    logging_steps=LOGGING_STEPS,
    logging_first_step=True,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=2,
    dataloader_num_workers=8,  # Increased for faster data loading
    gradient_checkpointing=False,  # Disabled for speed (DGX Spark has 128GB memory)
    seed=42,
    report_to="none",
)

print("\n✓ SFTConfig created!")
print("  Note: gradient_checkpointing=False for faster training (requires ~60-70GB memory)")

Training configuration:
  Total examples: 120,000
  Batch size: 8 x 2 = 16
  Estimated total steps: 7,500

✓ SFTConfig created!
  Note: gradient_checkpointing=False for faster training (requires ~60-70GB memory)


## 8. Create Trainer and Start Training

In [8]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=formatted_dataset,
    args=sft_config,
)

print("✓ Trainer created!")
print(f"\nStarting Full Fine-Tuning...")
print("=" * 60)

Unsloth: Sample packing skipped (custom data collator detected).
✓ Trainer created!

Starting Full Fine-Tuning...


In [9]:
import time

start_time = time.time()

trainer_stats = trainer.train()

elapsed_time = time.time() - start_time
hours, remainder = divmod(elapsed_time, 3600)
minutes, seconds = divmod(remainder, 60)

print("\n" + "=" * 60)
print("Full Fine-Tuning Complete!")
print("=" * 60)
print(f"\nTraining time: {int(hours)}h {int(minutes)}m {int(seconds)}s")
print(f"Final loss: {trainer_stats.training_loss:.4f}")
print(f"Total steps: {trainer_stats.global_step}")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 120,000 | Num Epochs = 1 | Total steps = 7,500
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 7,615,616,512 of 7,615,616,512 (100.00% trained)


Step,Training Loss
1,2.950600
50,1.957100
100,0.506500
150,0.484100
200,0.476700
250,0.476300
300,0.481500
350,0.473300
400,0.473500
450,0.475700



Full Fine-Tuning Complete!

Training time: 8h 35m 59s
Final loss: 0.4536
Total steps: 7500


## 9. Save the Full Model

In [10]:
model_path = f"{OUTPUT_DIR}/final"

print(f"Saving full fine-tuned model to: {model_path}")

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Check saved files
import os
saved_files = os.listdir(model_path)
total_size = sum(os.path.getsize(os.path.join(model_path, f)) for f in saved_files if os.path.isfile(os.path.join(model_path, f)))

print(f"\nSaved files:")
for f in sorted(saved_files)[:10]:  # Show first 10 files
    path = os.path.join(model_path, f)
    if os.path.isfile(path):
        size = os.path.getsize(path)
        print(f"  {f}: {size / 1e9:.2f} GB" if size > 1e9 else f"  {f}: {size / 1e6:.2f} MB")

print(f"\nTotal model size: {total_size / 1e9:.2f} GB")
print(f"\n✓ Full model saved!")

Saving full fine-tuned model to: ./checkpoints/qwen7b-ag-news-full-unsloth/final

Saved files:
  added_tokens.json: 0.00 MB
  chat_template.jinja: 0.00 MB
  config.json: 0.00 MB
  generation_config.json: 0.00 MB
  merges.txt: 1.67 MB
  model-00001-of-00004.safetensors: 4.88 GB
  model-00002-of-00004.safetensors: 4.93 GB
  model-00003-of-00004.safetensors: 4.33 GB
  model-00004-of-00004.safetensors: 1.09 GB
  model.safetensors.index.json: 0.03 MB

Total model size: 15.25 GB

✓ Full model saved!


## 10. Quick Evaluation

In [11]:
FastLanguageModel.for_inference(model)

SYSTEM_PROMPT = """You are a news article classifier. Categorize into: World, Sports, Business, or Sci/Tech.
Respond with JSON: {"category": "<category>"}"""

test_articles = [
    ("The Federal Reserve announced a quarter-point interest rate cut.", "Business"),
    ("Scientists at CERN discovered a new subatomic particle.", "Sci/Tech"),
    ("The Lakers defeated the Celtics 112-108 in overtime.", "Sports"),
    ("The UN Security Council voted to impose new sanctions.", "World"),
]

print("Testing full fine-tuned model:")
print("=" * 60)

correct = 0
for article, expected in test_articles:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Classify: {article}"},
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True,
    ).to(model.device)
    
    outputs = model.generate(
        inputs,
        max_new_tokens=50,
        temperature=0.0,
        do_sample=False,
    )
    
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    is_correct = expected.lower() in response.lower()
    if is_correct:
        correct += 1
    
    print(f"\nArticle: {article[:50]}...")
    print(f"Expected: {expected}")
    print(f"Response: {response.strip()}")
    print(f"Status: {'✓' if is_correct else '✗'}")

print(f"\n" + "=" * 60)
print(f"Quick test accuracy: {correct}/{len(test_articles)} ({100*correct/len(test_articles):.0f}%)")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Testing full fine-tuned model:

Article: The Federal Reserve announced a quarter-point inte...
Expected: Business
Response: {"category": "Business"}
Status: ✓

Article: Scientists at CERN discovered a new subatomic part...
Expected: Sci/Tech
Response: {"category": "Sci/Tech"}
Status: ✓

Article: The Lakers defeated the Celtics 112-108 in overtim...
Expected: Sports
Response: {"category": "Sports"}
Status: ✓

Article: The UN Security Council voted to impose new sancti...
Expected: World
Response: {"category": "World"}
Status: ✓

Quick test accuracy: 4/4 (100%)


## Conclusions

### Training Results

| Metric | Value |
|--------|-------|
| **Training Time** | 8h 35m 59s |
| **Avg Training Loss** | 0.4536 |
| **Total Steps** | 7,500 |
| **Trainable Parameters** | 7.6B (100%) |
| **Model Size** | 15.25 GB |
| **Training Speed** | ~0.24 it/s |
| **GPU Memory** | ~70 GB |

### Quick Validation Results

| Test Article | Expected | Predicted | Status |
|--------------|----------|-----------|--------|
| Federal Reserve interest rate cut | Business | Business | ✓ |
| CERN subatomic particle discovery | Sci/Tech | Sci/Tech | ✓ |
| Lakers vs Celtics game | Sports | Sports | ✓ |
| UN Security Council sanctions | World | World | ✓ |

**Quick test accuracy: 4/4 (100%)**

### Comparison: Full Fine-Tuning vs LoRA vs QLoRA

| Aspect | QLoRA | LoRA | Full Fine-Tuning |
|--------|-------|------|------------------|
| **Parameters Trained** | 40M (0.5%) | 40M (0.5%) | 7.6B (100%) |
| **Training Speed** | ~0.35 it/s | ~0.36 it/s | ~0.24 it/s |
| **Training Time** | 5h 58m | 5h 51m | 8h 36m |
| **Avg Training Loss** | 0.4625 | 0.4600 | 0.4536 |
| **Memory Usage** | ~15 GB | ~25 GB | ~70 GB |
| **Output Size** | 177 MB | 177 MB | 15.25 GB |

### Key Insights

1. **Full fine-tuning achieves the lowest loss** (0.4536 vs 0.4600/0.4625):
   - Training all 7.6B parameters allows more complete model adaptation
   - ~1.4% lower average loss than LoRA, ~1.9% lower than QLoRA
   - Difference is modest but may translate to slightly better accuracy

2. **Training is slower but not drastically** (0.24 it/s vs 0.35 it/s):
   - Full fine-tuning is ~33% slower than LoRA/QLoRA
   - Memory bandwidth is the bottleneck for all methods on DGX Spark
   - 8.5h is reasonable for training all 7.6B parameters

3. **Memory usage is significant but manageable**:
   - ~70 GB fits comfortably in DGX Spark's 128 GB
   - `adamw_8bit` optimizer saves ~15 GB vs standard AdamW
   - `gradient_checkpointing=False` trades memory for speed

4. **Output is a full model, not an adapter**:
   - 15.25 GB model can be deployed standalone
   - No need to load base model + adapter separately
   - Can be further quantized (GGUF, AWQ, etc.) for deployment

### Memory Breakdown

| Component | Size |
|-----------|------|
| Model weights (BF16) | ~15 GB |
| Gradients (BF16) | ~15 GB |
| Optimizer states (8-bit) | ~15 GB |
| Activations | ~15-25 GB |
| **Total** | **~60-70 GB** |

### DGX Spark Optimizations Impact

| Optimization | Impact |
|--------------|--------|
| `gradient_checkpointing=False` | +50% speed (0.16 → 0.24 it/s) |
| `BATCH_SIZE=8` | Better GPU utilization |
| `adamw_8bit` | -15 GB memory |
| `dataloader_num_workers=8` | Faster data loading |

### Recommendation

**When to use Full Fine-Tuning:**
- Maximum quality is required
- You have sufficient memory (60+ GB)
- You want a standalone model (no adapter dependencies)
- Training time is not critical

**When to use LoRA/QLoRA instead:**
- Rapid iteration needed (5-6h vs 8.5h)
- Memory constrained environments
- You want small adapter files for easy switching
- Loss difference (~1-2%) is acceptable

### Next Steps

1. **Full Evaluation**: Run comprehensive test on AG News test set (7,600 samples)
2. **Compare Accuracy**: Verify if 1.4% lower loss translates to better accuracy than QLoRA (95.14%)
3. **Quantization**: Export model to GGUF/AWQ for efficient deployment